In [95]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from xgboost import XGBClassifier, XGBRegressor
import xgboost as xgb

## Importing data

In [96]:
features_num = [
    'Total_flux', 'Peak_flux', 'FUV_flux_corr', 'NUV_flux_corr', 'g_flux_corr', 'r_flux_corr', 'i_flux_corr',
    'z_flux_corr', 'y_flux_corr', 'g_hsc_flux_corr', 'r_hsc_flux_corr', 'i_hsc_flux_corr', 'z_hsc_flux_corr',
    'y_hsc_flux_corr', 'nb921_hsc_flux_corr', 'J_flux_corr', 'K_flux_corr', 'ch1_swire_flux_corr', 
    'ch2_swire_flux_corr', 'ch3_swire_flux_corr', 'ch4_swire_flux_corr', 'ch1_servs_flux_corr', 'ch2_servs_flux_corr',
    'F_MIPS_24', 'F_PACS_100', 'F_PACS_160', 'F_SPIRE_250', 'F_SPIRE_350', 'F_SPIRE_500', 'Z_BEST', 'EBV'
    ]
statmorph_features = ['ellipticity_centroid', 'elongation_centroid', 
                      'r20', 'r50', 'r80', 'C', 'A', 'S', 'M20', 'Gini',]

#statmorph_features = ['intensity']

#statmorph_features = ['r20']
y_column = "Classification"

classes = ['jet-mode radio AGN/low-excitation radio galaxy', 'quasar-like radio AGN / high-excitation radio galaxy', 
           'radio-quiet AGN', 'star-forming galaxy']

In [97]:
# Loading data, Only selecting ELais-N1, since we do not have statmorph data on the others
data = pd.read_csv("../../../Data/Fangyou_data/Cleaned/Elais-N1.csv")
data['Source_Name'] = data['Source_Name']

# Loading statmorph data
statmorph_data = pd.read_csv("../../../General/Fangyou/HSC_images/statmorph.dat", sep=' ')
statmorph_data.ID = statmorph_data.ID.str[0:-5]

In [98]:
statmorph_data.columns

Index(['ID', 'xc_centroid', 'yc_centroid', 'ellipticity_centroid',
       'elongation_centroid', 'orientation_centroid', 'xc_asymmetry',
       'yc_asymmetry', 'ellipticity_asymmetry', 'elongation_asymmetry',
       'orientation_asymmetry', 'r20', 'r50', 'r80', 'rhalf_circ',
       'rhalf_ellip', 'rmax_circ', 'rmax_ellip', 'rpetro_circ', 'rpetro_ellip',
       'C', 'A', 'S', 'M20', 'Gini', 'F(G,M20)', 'S(G,M20)', 'deviation',
       'shape_asymmetry', 'outer_asymmetry', 'multimode', 'sn_per_pixel',
       'flux_circ', 'flux_ellip', 'intensity', 'sersic_xc', 'sersic_yc',
       'sersic_amplitude', 'sersic_ellip', 'sersic_n', 'sersic_rhalf',
       'sersic_theta', 'sky_mean', 'sky_median', 'sky_sigma', 'nx_stamp',
       'ny_stamp', 'xmax_stamp', 'xmin_stamp', 'ymax_stamp', 'ymin_stamp',
       'flag', 'flag_sersic'],
      dtype='object')

In [99]:
pd.DataFrame(statmorph_data[['flag', 'flag_sersic']].value_counts(), columns=['count'])

count
flag flag_sersic       
1.0  1.0          13790
0.0  0.0           5872
     1.0           2883
1.0  0.0            976

In [100]:
statmorph_data = statmorph_data[statmorph_data['flag']==0]
statmorph_data = statmorph_data[statmorph_data['flag_sersic']==0]

# Only selecting ids where we have statmorph data for
data = data[data['Source_Name'].isin(statmorph_data.ID)]

In [101]:
combined_data = data.merge(statmorph_data, left_on='Source_Name', right_on='ID')
combined_data = combined_data[np.append(features_num, statmorph_features)]

In [102]:
# Normal data without statmorph features
X_normal = data[features_num]
y = data[y_column]

# Label encoding y
le = LabelEncoder()
labels = np.unique(y.astype(str))
y = le.fit_transform(y)

In [103]:
combined_data = combined_data[np.append(features_num, statmorph_features)]

In [104]:
X_combined_train, X_combined_test, y_train, y_test =\
    train_test_split(combined_data, y, train_size=0.7, stratify=y, random_state=42)
X_combined_test, X_combined_val, y_test, y_val = train_test_split(X_combined_test, y_test, train_size=0.7, stratify=y_test, random_state=42)

In [105]:
# Normal data
X_normal_train, X_normal_test, y_train, y_test =\
    train_test_split(X_normal, y, train_size=0.7, stratify=y, random_state=42)
X_normal_test, X_normal_val, y_test, y_val = train_test_split(X_normal_test, y_test, train_size=0.7, stratify=y_test, random_state=42)

# Rnning model

## Normal accuracy

In [75]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

In [76]:
model = XGBClassifier(use_label_encoder=False, 
                      max_depth=4, 
                      #reg_alpha=1,   
                      #min_child_weight=0.5, 
                      #reg_lambda=5,
                      #subsample=0.5,
                      eta=0.3, # Remember to reduce to 0.1 or 0.05 for better results
                      tree_method='gpu_hist', # exact is more precies, but this is much faster
                      gpu_id=0,
                      objective='multi:softprob',
                      eval_metric =['merror'],
                      nthread=8,
                      n_estimators=10000,
                      )

In [77]:
bst = model.fit(X_normal_train, y_train, 
                eval_set=[(X_normal_val, y_val)], 
                early_stopping_rounds=400,
                verbose=True, 
                #eval_metric=xgb_recall
                #sample_weight=classes_weights,
               ) # Maybe remove weight

[0]	validation_0-merror:0.10964
[1]	validation_0-merror:0.10775
[2]	validation_0-merror:0.10397
[3]	validation_0-merror:0.10208
[4]	validation_0-merror:0.10586


C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[5]	validation_0-merror:0.09452
[6]	validation_0-merror:0.09074
[7]	validation_0-merror:0.09830
[8]	validation_0-merror:0.09263
[9]	validation_0-merror:0.08885
[10]	validation_0-merror:0.09074
[11]	validation_0-merror:0.08696
[12]	validation_0-merror:0.08318
[13]	validation_0-merror:0.08507
[14]	validation_0-merror:0.07751
[15]	validation_0-merror:0.07751
[16]	validation_0-merror:0.07751
[17]	validation_0-merror:0.08128
[18]	validation_0-merror:0.07939
[19]	validation_0-merror:0.07939
[20]	validation_0-merror:0.07939
[21]	validation_0-merror:0.07939
[22]	validation_0-merror:0.07561
[23]	validation_0-merror:0.07372
[24]	validation_0-merror:0.07372
[25]	validation_0-merror:0.07561
[26]	validation_0-merror:0.07561
[27]	validation_0-merror:0.07561
[28]	validation_0-merror:0.07561
[29]	validation_0-merror:0.07372
[30]	validation_0-merror:0.07561
[31]	validation_0-merror:0.07372
[32]	validation_0-merror:0.07372
[33]	validation_0-merror:0.07183
[34]	validation_0-merror:0.07183
[35]	validation

[249]	validation_0-merror:0.07183
[250]	validation_0-merror:0.07183
[251]	validation_0-merror:0.07372
[252]	validation_0-merror:0.07372
[253]	validation_0-merror:0.07372
[254]	validation_0-merror:0.07372
[255]	validation_0-merror:0.07183
[256]	validation_0-merror:0.07183
[257]	validation_0-merror:0.06994
[258]	validation_0-merror:0.06994
[259]	validation_0-merror:0.06994
[260]	validation_0-merror:0.07183
[261]	validation_0-merror:0.06994
[262]	validation_0-merror:0.06994
[263]	validation_0-merror:0.07183
[264]	validation_0-merror:0.07183
[265]	validation_0-merror:0.06994
[266]	validation_0-merror:0.07183
[267]	validation_0-merror:0.07183
[268]	validation_0-merror:0.07183
[269]	validation_0-merror:0.06994
[270]	validation_0-merror:0.06994
[271]	validation_0-merror:0.06994
[272]	validation_0-merror:0.07183
[273]	validation_0-merror:0.07183
[274]	validation_0-merror:0.07183
[275]	validation_0-merror:0.07183
[276]	validation_0-merror:0.07183
[277]	validation_0-merror:0.06994
[278]	validati

[490]	validation_0-merror:0.07183
[491]	validation_0-merror:0.07183
[492]	validation_0-merror:0.07183
[493]	validation_0-merror:0.07183
[494]	validation_0-merror:0.07183
[495]	validation_0-merror:0.07183
[496]	validation_0-merror:0.07183
[497]	validation_0-merror:0.07183
[498]	validation_0-merror:0.07183
[499]	validation_0-merror:0.07183


In [78]:
y_pred = model.predict(X_normal_test)
print(classification_report(y_test, y_pred, target_names=labels, digits=4))

                                                      precision    recall  f1-score   support

      jet-mode radio AGN/low-excitation radio galaxy     0.8667    0.6393    0.7358       122
quasar-like radio AGN / high-excitation radio galaxy     0.3333    0.0769    0.1250        13
                                     radio-quiet AGN     0.7931    0.7500    0.7709        92
                                 star-forming galaxy     0.9383    0.9821    0.9597      1006

                                            accuracy                         0.9213      1233
                                           macro avg     0.7328    0.6121    0.6479      1233
                                        weighted avg     0.9140    0.9213    0.9147      1233



## More features

In [79]:
model = XGBClassifier(use_label_encoder=False, 
                      max_depth=4, 
                      #reg_alpha=1,   
                      #min_child_weight=0.5, 
                      #reg_lambda=5,
                      #subsample=0.5,
                      eta=0.3, # Remember to reduce to 0.1 or 0.05 for better results
                      tree_method='gpu_hist', # exact is more precies, but this is much faster
                      gpu_id=0,
                      objective='multi:softprob',
                      eval_metric =['merror'],
                      nthread=8,
                      n_estimators=10000,
                      )

In [80]:
len(X_combined_train)

4110

In [81]:
bst = model.fit(X_combined_train, y_train, 
                eval_set=[(X_combined_val, y_val)], 
                early_stopping_rounds=100,
                verbose=True, 
                #eval_metric=xgb_recall
                #sample_weight=classes_weights,
               ) # Maybe remove weight

[0]	validation_0-merror:0.12098
[1]	validation_0-merror:0.10964
[2]	validation_0-merror:0.11531
[3]	validation_0-merror:0.10964
[4]	validation_0-merror:0.10586


C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[5]	validation_0-merror:0.10586
[6]	validation_0-merror:0.09830
[7]	validation_0-merror:0.09263
[8]	validation_0-merror:0.10019
[9]	validation_0-merror:0.09830
[10]	validation_0-merror:0.09641
[11]	validation_0-merror:0.09263
[12]	validation_0-merror:0.08885
[13]	validation_0-merror:0.08696
[14]	validation_0-merror:0.08318
[15]	validation_0-merror:0.09074
[16]	validation_0-merror:0.08885
[17]	validation_0-merror:0.08696
[18]	validation_0-merror:0.08128
[19]	validation_0-merror:0.08318
[20]	validation_0-merror:0.08128
[21]	validation_0-merror:0.07939
[22]	validation_0-merror:0.07372
[23]	validation_0-merror:0.07372
[24]	validation_0-merror:0.07561
[25]	validation_0-merror:0.07183
[26]	validation_0-merror:0.07183
[27]	validation_0-merror:0.06994
[28]	validation_0-merror:0.06805
[29]	validation_0-merror:0.06805
[30]	validation_0-merror:0.06616
[31]	validation_0-merror:0.06805
[32]	validation_0-merror:0.06616
[33]	validation_0-merror:0.06616
[34]	validation_0-merror:0.06427
[35]	validation

In [82]:
y_pred = model.predict(X_combined_test)
print(classification_report(y_test, y_pred, target_names=labels, digits=4))

                                                      precision    recall  f1-score   support

      jet-mode radio AGN/low-excitation radio galaxy     0.8621    0.6148    0.7177       122
quasar-like radio AGN / high-excitation radio galaxy     0.5000    0.1538    0.2353        13
                                     radio-quiet AGN     0.8072    0.7283    0.7657        92
                                 star-forming galaxy     0.9339    0.9831    0.9579      1006

                                            accuracy                         0.9189      1233
                                           macro avg     0.7758    0.6200    0.6691      1233
                                        weighted avg     0.9128    0.9189    0.9121      1233



## Just Statmorph

In [106]:
X_statmorph_train = X_combined_train.drop(columns=features_num)
X_statmorph_test = X_combined_test.drop(columns=features_num)
X_statmorph_val = X_combined_val.drop(columns=features_num)

In [107]:
model = XGBClassifier(use_label_encoder=False, 
                      max_depth=4, 
                      #reg_alpha=1,   
                      #min_child_weight=0.5, 
                      #reg_lambda=5,
                      #subsample=0.5,
                      eta=0.3, # Remember to reduce to 0.1 or 0.05 for better results
                      tree_method='gpu_hist', # exact is more precies, but this is much faster
                      gpu_id=0,
                      objective='multi:softprob',
                      eval_metric =['merror'],
                      nthread=8,
                      n_estimators=10000,
                      )

In [108]:
bst = model.fit(X_statmorph_train, y_train, 
                eval_set=[(X_statmorph_val, y_val)], 
                early_stopping_rounds=100,
                verbose=True, 
                #eval_metric=xgb_recall
                #sample_weight=classes_weights,
               ) # Maybe remove weight

[0]	validation_0-merror:0.15123
[1]	validation_0-merror:0.15501
[2]	validation_0-merror:0.15501
[3]	validation_0-merror:0.15690
[4]	validation_0-merror:0.15312


C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[5]	validation_0-merror:0.15312
[6]	validation_0-merror:0.15123
[7]	validation_0-merror:0.15123
[8]	validation_0-merror:0.15501
[9]	validation_0-merror:0.15312
[10]	validation_0-merror:0.15312
[11]	validation_0-merror:0.15501
[12]	validation_0-merror:0.15312
[13]	validation_0-merror:0.14934
[14]	validation_0-merror:0.14934
[15]	validation_0-merror:0.14745
[16]	validation_0-merror:0.14556
[17]	validation_0-merror:0.14556
[18]	validation_0-merror:0.14934
[19]	validation_0-merror:0.14745
[20]	validation_0-merror:0.14934
[21]	validation_0-merror:0.14745
[22]	validation_0-merror:0.14934
[23]	validation_0-merror:0.14934
[24]	validation_0-merror:0.14934
[25]	validation_0-merror:0.15312
[26]	validation_0-merror:0.15312
[27]	validation_0-merror:0.15501
[28]	validation_0-merror:0.15501
[29]	validation_0-merror:0.15690
[30]	validation_0-merror:0.15879
[31]	validation_0-merror:0.15879
[32]	validation_0-merror:0.15879
[33]	validation_0-merror:0.15879
[34]	validation_0-merror:0.15879
[35]	validation

In [109]:
y_pred = model.predict(X_statmorph_test)
print(classification_report(y_test, y_pred, target_names=labels, digits=4))

                                                      precision    recall  f1-score   support

      jet-mode radio AGN/low-excitation radio galaxy     0.5897    0.1885    0.2857       122
quasar-like radio AGN / high-excitation radio galaxy     0.0000    0.0000    0.0000        13
                                     radio-quiet AGN     0.7755    0.4130    0.5390        92
                                 star-forming galaxy     0.8638    0.9831    0.9196      1006

                                            accuracy                         0.8516      1233
                                           macro avg     0.5573    0.3962    0.4361      1233
                                        weighted avg     0.8210    0.8516    0.8188      1233



C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1327: Unde

In [91]:
from flaml import AutoML
automl = AutoML()

C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flaml\searcher\blendsearch.py:14: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
  from ray.tune.suggest import Searcher
C:\Users\jespe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flaml\searcher\blendsearch.py:15: DeprecationWarning: The module `ray.tune.suggest.optuna` has been moved to `ray.tune.search.optuna` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest.optuna` with `ray.tune.search.optuna`.
  from ray.tune.suggest.optuna import OptunaSe

In [92]:
import warnings

estimators = ['xgb_limitdepth']
#estimators = ['extra_tree']

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    automl.fit(X_statmorph_train, y_train, task="classification", metric='accuracy',
                                #X_val=X_val , y_val=y_val,
                                #ensemble=True,
                                #estimator_list=estimators, 
                                time_budget=600, n_jobs=8,
                                eval_method='cv',
                                #log_file_name='general_extra_features3.log',
                                #starting_points=automl.best_config_per_estimator
              )

[flaml.automl: 09-30 15:14:08] {2390} INFO - task = classification
[flaml.automl: 09-30 15:14:08] {2392} INFO - Data split method: stratified
[flaml.automl: 09-30 15:14:08] {2396} INFO - Evaluation method: cv
[flaml.automl: 09-30 15:14:08] {2465} INFO - Minimizing error metric: 1-accuracy
[flaml.automl: 09-30 15:14:09] {2605} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 09-30 15:14:09] {2897} INFO - iteration 0, current learner lgbm
[flaml.automl: 09-30 15:14:09] {3025} INFO - Estimated sufficient time budget=1180s. Estimated necessary time budget=29s.
[flaml.automl: 09-30 15:14:09] {3072} INFO -  at 1.8s,	estimator lgbm's best error=0.1847,	best estimator lgbm's best error=0.1847
[flaml.automl: 09-30 15:14:09] {2897} INFO - iteration 1, current learner lgbm
[flaml.automl: 09-30 15:14:09] {3072} INFO -  at 1.9s,	estimator lgbm's best error=0.1847,	best estimator lgbm's best error=0.1847
[flaml.aut

[flaml.automl: 09-30 15:14:16] {3072} INFO -  at 8.6s,	estimator xgboost's best error=0.1620,	best estimator xgboost's best error=0.1620
[flaml.automl: 09-30 15:14:16] {2897} INFO - iteration 36, current learner xgboost
[flaml.automl: 09-30 15:14:16] {3072} INFO -  at 8.7s,	estimator xgboost's best error=0.1620,	best estimator xgboost's best error=0.1620
[flaml.automl: 09-30 15:14:16] {2897} INFO - iteration 37, current learner xgboost
[flaml.automl: 09-30 15:14:16] {3072} INFO -  at 8.9s,	estimator xgboost's best error=0.1620,	best estimator xgboost's best error=0.1620
[flaml.automl: 09-30 15:14:16] {2897} INFO - iteration 38, current learner xgboost
[flaml.automl: 09-30 15:14:16] {3072} INFO -  at 9.1s,	estimator xgboost's best error=0.1620,	best estimator xgboost's best error=0.1620
[flaml.automl: 09-30 15:14:16] {2897} INFO - iteration 39, current learner rf
[flaml.automl: 09-30 15:14:17] {3072} INFO -  at 9.4s,	estimator rf's best error=0.1637,	best estimator xgboost's best error=

[flaml.automl: 09-30 15:14:25] {3072} INFO -  at 17.9s,	estimator xgboost's best error=0.1608,	best estimator xgboost's best error=0.1608
[flaml.automl: 09-30 15:14:25] {2897} INFO - iteration 74, current learner catboost
[flaml.automl: 09-30 15:14:26] {3072} INFO -  at 18.6s,	estimator catboost's best error=0.1837,	best estimator xgboost's best error=0.1608
[flaml.automl: 09-30 15:14:26] {2897} INFO - iteration 75, current learner xgboost
[flaml.automl: 09-30 15:14:26] {3072} INFO -  at 18.8s,	estimator xgboost's best error=0.1608,	best estimator xgboost's best error=0.1608
[flaml.automl: 09-30 15:14:26] {2897} INFO - iteration 76, current learner rf
[flaml.automl: 09-30 15:14:27] {3072} INFO -  at 19.5s,	estimator rf's best error=0.1618,	best estimator xgboost's best error=0.1608
[flaml.automl: 09-30 15:14:27] {2897} INFO - iteration 77, current learner catboost
[flaml.automl: 09-30 15:14:27] {3072} INFO -  at 20.0s,	estimator catboost's best error=0.1837,	best estimator xgboost's be

[flaml.automl: 09-30 15:14:38] {2897} INFO - iteration 111, current learner lgbm
[flaml.automl: 09-30 15:14:38] {3072} INFO -  at 30.8s,	estimator lgbm's best error=0.1667,	best estimator xgboost's best error=0.1608
[flaml.automl: 09-30 15:14:38] {2897} INFO - iteration 112, current learner extra_tree
[flaml.automl: 09-30 15:14:38] {3072} INFO -  at 31.1s,	estimator extra_tree's best error=0.1740,	best estimator xgboost's best error=0.1608
[flaml.automl: 09-30 15:14:38] {2897} INFO - iteration 113, current learner xgboost
[flaml.automl: 09-30 15:14:39] {3072} INFO -  at 31.2s,	estimator xgboost's best error=0.1608,	best estimator xgboost's best error=0.1608
[flaml.automl: 09-30 15:14:39] {2897} INFO - iteration 114, current learner extra_tree
[flaml.automl: 09-30 15:14:39] {3072} INFO -  at 31.6s,	estimator extra_tree's best error=0.1740,	best estimator xgboost's best error=0.1608
[flaml.automl: 09-30 15:14:39] {2897} INFO - iteration 115, current learner catboost
[flaml.automl: 09-30 

[flaml.automl: 09-30 15:14:57] {2897} INFO - iteration 149, current learner lgbm
[flaml.automl: 09-30 15:14:58] {3072} INFO -  at 50.2s,	estimator lgbm's best error=0.1659,	best estimator rf's best error=0.1599
[flaml.automl: 09-30 15:14:58] {2897} INFO - iteration 150, current learner lgbm
[flaml.automl: 09-30 15:14:58] {3072} INFO -  at 50.3s,	estimator lgbm's best error=0.1659,	best estimator rf's best error=0.1599
[flaml.automl: 09-30 15:14:58] {2897} INFO - iteration 151, current learner rf
[flaml.automl: 09-30 15:14:59] {3072} INFO -  at 51.9s,	estimator rf's best error=0.1599,	best estimator rf's best error=0.1599
[flaml.automl: 09-30 15:14:59] {2897} INFO - iteration 152, current learner xgboost
[flaml.automl: 09-30 15:14:59] {3072} INFO -  at 52.0s,	estimator xgboost's best error=0.1608,	best estimator rf's best error=0.1599
[flaml.automl: 09-30 15:14:59] {2897} INFO - iteration 153, current learner catboost
[flaml.automl: 09-30 15:15:00] {3072} INFO -  at 52.5s,	estimator cat

[flaml.automl: 09-30 15:15:12] {3072} INFO -  at 64.9s,	estimator xgb_limitdepth's best error=0.1618,	best estimator rf's best error=0.1599
[flaml.automl: 09-30 15:15:12] {2897} INFO - iteration 187, current learner catboost
[flaml.automl: 09-30 15:15:13] {3072} INFO -  at 65.4s,	estimator catboost's best error=0.1837,	best estimator rf's best error=0.1599
[flaml.automl: 09-30 15:15:13] {2897} INFO - iteration 188, current learner xgb_limitdepth
[flaml.automl: 09-30 15:15:13] {3072} INFO -  at 65.5s,	estimator xgb_limitdepth's best error=0.1618,	best estimator rf's best error=0.1599
[flaml.automl: 09-30 15:15:13] {2897} INFO - iteration 189, current learner rf
[flaml.automl: 09-30 15:15:14] {3072} INFO -  at 66.5s,	estimator rf's best error=0.1599,	best estimator rf's best error=0.1599
[flaml.automl: 09-30 15:15:14] {2897} INFO - iteration 190, current learner catboost
[flaml.automl: 09-30 15:15:14] {3072} INFO -  at 66.9s,	estimator catboost's best error=0.1837,	best estimator rf's be